In [ ]:
use_gdrive = 'yes'  #@param ["yes", "no"]

if use_gdrive == 'yes':
  from google.colab import drive
  drive.mount('/gdrive')
  root = '/gdrive/My Drive/cifar10'
  import os
  if not os.path.isdir(root):
    os.mkdir(root)
  os.chdir(root)
  print(f'\nChanged CWD to "{root}"')
else:
  from IPython import display
  display.display(display.HTML(
      '<h1 style="color:red">CHANGES NOT PERSISTED</h1>'))

In [1]:
!pip install --upgrade transformers
!pip install --upgrade datasets
!pip install --upgrade huggingface_hub
!pip install evaluate
!pip install accelerate
!pip install ptflops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.31.0, but you have requests 2.32.3 which is incompatible.


In [ ]:
from datasets import load_dataset
from huggingface_hub import login
from transformers import ViTImageProcessor,AutoProcessor
from torchvision.transforms import (CenterCrop,
                                    Compose,
                                    Normalize,
                                    RandomHorizontalFlip,
                                    RandomResizedCrop,
                                    Resize,
                                    ToTensor)
from torch.utils.data import DataLoader
import torch
from transformers import ViTForImageClassification, AutoModelForImageClassification
from transformers import TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, top_k_accuracy_score
import numpy as np
from evaluate import evaluator
from ptflops import get_model_complexity_info

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
def train_transforms(examples):
    examples['pixel_values'] = [_train_transforms(image.convert("RGB")) for image in examples['img']]
    return examples

def val_transforms(examples):
    examples['pixel_values'] = [_val_transforms(image.convert("RGB")) for image in examples['img']]
    return examples

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return dict(accuracy=accuracy_score(predictions, labels))

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [2]:
from datasets import load_dataset

In [3]:
train, test = load_dataset('cifar10', split=["train", "test"])
#splits = train.train_test_split(test_size=0.2)
#train_ds = splits['train']
#val_ds = splits['test']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [7]:
train.features

{'img': Image(mode=None, decode=True, id=None),
 'label': ClassLabel(names=['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck'], id=None)}

In [ ]:
id2label = {id:label for id, label in enumerate(train.features['label'].names)}
label2id = {label:id for id,label in id2label.items()}

model_name = 'google/vit-base-patch16-224-in21k' #@param ["google/vit-base-patch16-224-in21k", "edadaltocg/vit_base_patch16_224_in21k_ft_cifar100"]

processor = AutoProcessor.from_pretrained(model_name)

image_mean, image_std = processor.image_mean, processor.image_std
size = processor.size["height"]

normalize = Normalize(mean=image_mean, std=image_std)

_train_transforms = Compose(
    [
        RandomResizedCrop(size),
        RandomHorizontalFlip(),
        ToTensor(),
        normalize,
    ]
)

_val_transforms = Compose(
    [
        Resize(size),
        CenterCrop(size),
        ToTensor(),
        normalize,
    ]
)

train.set_transform(train_transforms)
#val_ds.set_transform(val_transforms)
test.set_transform(val_transforms)


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForImageClassification.from_pretrained(model_name, id2label=id2label, label2id=label2id, ignore_mismatched_sizes=True)

model.to(device)

metric_name = "accuracy"

In [ ]:
args = TrainingArguments(
    f"{model_name}-cifar-10",
    save_strategy="epoch",
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    fp16=True,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    logging_dir='logs',
    remove_unused_columns=False,
    seed=42,
    gradient_accumulation_steps=4,
    lr_scheduler_type="linear",
)

trainer = Trainer(
    model,
    args,
    train_dataset=train,
    eval_dataset=test,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    tokenizer=processor,
)

macs, params = get_model_complexity_info(model, (3, 224, 224), as_strings=False, print_per_layer_stat=False, verbose=False)
model_gflops = round(2 * int(macs) / 1000000000, 1)
print(f'\t- Parameters: {params}')
print(f'\t- GFLOPs: {model_gflops}')

train_results = trainer.train()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

metrics = trainer.evaluate()
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


	- Parameters: 85806346
	- GFLOPs: 35.2


Epoch,Training Loss,Validation Loss,Accuracy
0,No log,0.183748,0.980400
1,0.619300,0.111032,0.985000
2,0.280600,0.094309,0.986500


***** train metrics *****
  epoch                    =          2.99
  total_flos               = 10806650438GF
  train_loss               =        0.4186
  train_runtime            =    0:33:07.92
  train_samples_per_second =        75.455
  train_steps_per_second   =         0.589


***** eval metrics *****
  epoch                   =       2.99
  eval_accuracy           =     0.9865
  eval_loss               =     0.0943
  eval_runtime            = 0:00:52.20
  eval_samples_per_second =     191.57
  eval_steps_per_second   =      5.996


CommitInfo(commit_url='https://huggingface.co/Tsomaros/vit-base-patch16-224-in21k-cifar-10/commit/a8d68b6922c0ec24da7ceb0f0a150047e6f35bac', commit_message='vit-base-patch16-224-in21k-cifar-10', commit_description='', oid='a8d68b6922c0ec24da7ceb0f0a150047e6f35bac', pr_url=None, pr_revision=None, pr_num=None)